In [1]:
from itertools import chain

import pandas as pd

from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import add_geo_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data
from ddeserts.load import load_charter_cities

In [2]:
# load the data
pre_filter = lambda line: ('California' in line and 'CDP' not in line)

df = load_cvap_data('Place', pre_filter=pre_filter)
len(df)

485

In [3]:
# add geo columns and post-filter
add_geo_columns(df)
df = df[(df['state'] == 'California') & df['geotype'].isin(['city', 'town'])]
len(df)

482

In [4]:
# add charter city data
charter_cities = load_charter_cities()
df['geosubtype'] = 'general law'
df.loc[df['name'].isin(charter_cities), 'geosubtype'] = 'charter'

In [5]:
# add stat columns and sort
add_all_stat_columns(df)
df = with_columns_sorted(df)

In [6]:
# sort by common name of city (Ventura, not San Buenaventura)
df.sort_values(by=['name'], inplace=True)

In [7]:
df

geoid                        geoname   geosubtype  \
table line                                                                
Place 25130  16000US0600296      Adelanto city, California      charter   
      25156  16000US0600394  Agoura Hills city, California  general law   
      25221  16000US0600562       Alameda city, California      charter   
      25247  16000US0600674        Albany city, California      charter   
      25286  16000US0600884      Alhambra city, California      charter   
...                     ...                            ...          ...   
      44786  16000US0686930    Yountville city, California  general law   
      44799  16000US0686944         Yreka city, California  general law   
      44812  16000US0686972     Yuba City city, California  general law   
      44825  16000US0687042       Yucaipa city, California  general law   
      44838  16000US0687056  Yucca Valley town, California  general law   

            geotype          name       state  adu_est  adu_moe  cit_est  \
table line                                                                 
Place 25130    city      Adelanto  California    21995      653    29610   
      25156    city  Agoura Hills  California    16140      393    19320   
      25221    city       Alameda  California    62565      524    72155   
      25247    city        Albany  California    14665      241    16290   
      25286    city      Alhambra  California    70315      496    69830   
...             ...           ...         ...      ...      ...      ...   
      44786    city    Yountville  California     2760       81     2940   
      44799    city         Yreka  California     5660      180     7500   
      44812    city     Yuba City  California    49455      404    58480   
      44825    city       Yucaipa  California    40175      646    50875   
      44838    city  Yucca Valley  California    16730      502    21165   

             cit_moe  ...  prop_his_adu_dvap_est  prop_his_adu_dvap_moe  \
table line            ...                                                 
Place 25130      532  ...               0.265078               0.070049   
      25156      468  ...               0.243333               0.251120   
      25221      810  ...               0.152388               0.113577   
      25247      629  ...               0.281013               0.191676   
      25286      975  ...               0.149142               0.051519   
...              ...  ...                    ...                    ...   
      44786       41  ...               0.041667               0.480925   
      44799      107  ...               0.067308               0.395029   
      44812      771  ...               0.344544               0.050885   
      44825      470  ...               0.142425               0.109854   
      44838      199  ...               0.100149               0.195586   

             prop_ind_adu_dvap_est  prop_ind_adu_dvap_moe  \
table line                                                  
Place 25130               0.000000               2.121320   
      25156               0.000000              23.000000   
      25221               0.000000               0.699250   
      25247               0.000000               0.821591   
      25286               0.040816               0.661508   
...                            ...                    ...   
      44786               0.000000               1.979899   
      44799               0.000000               0.403313   
      44812               0.000000               0.442872   
      44825               0.062500               1.117735   
      44838               0.000000               0.754247   

             prop_pac_adu_dvap_est  prop_pac_adu_dvap_moe  \
table line                                                  
Place 25130               0.166667               2.091584   
      25156               0.000000              23.000000   
      25221               0.125000         

In [8]:
df.to_csv('output/ca-cities.csv')

In [9]:
cols = ['name', 'geosubtype'] + list(chain(*(
    [f'{pop}_est', f'{pop}_moe'] for pop in ('tot', 'adu', 'dvap', 'prop_adu_dvap'))))
dg = df[cols]
dg.to_csv('output/ca-cities-dvap.csv')

In [10]:
dg = df[['name', 'geosubtype', 'racial_disp_his_est', 'racial_disp_his_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_his_est'])[:30]

name geosubtype  racial_disp_his_est  \
table line                                                        
Place 33788           King City    charter            -0.185251   
      40860          San Rafael    charter            -0.134411   
      40340             Salinas    charter            -0.134153   
      40990         Santa Maria    charter            -0.134112   
      40522           Sand City    charter            -0.115367   
      39742            Richmond    charter            -0.113413   
      28900             Compton    charter            -0.112620   
      34984             Lindsay    charter            -0.102123   
      43928         Watsonville    charter            -0.101633   
      25546             Anaheim    charter            -0.098787   
      41354             Shafter    charter            -0.096745   
      33268           Inglewood    charter            -0.096371   
      29732  Desert Hot Springs    charter            -0.094919   
      37233                Napa    charter            -0.093939   
      40899           Santa Ana    charter            -0.093314   
      39651        Redwood City    charter            -0.088934   
      40912       Santa Barbara    charter            -0.085192   
      35400         Los Angeles    charter            -0.084399   
      41042          Santa Rosa    charter            -0.080334   
      39157         Porterville    charter            -0.079973   
      40756          San Marcos    charter            -0.077902   
      37740             Oakland    charter            -0.072734   
      43148              Tulare    charter            -0.072679   
      30915              Exeter    charter            -0.072302   
      36284              Merced    charter            -0.067492   
      25130            Adelanto    charter            -0.066996   
      40457      San Bernardino    charter            -0.066157   
      43720               Vista    charter            -0.064779   
      35309          Long Beach    charter            -0.062910   
      40795           San Mateo    charter            -0.058095   

             racial_disp_his_moe  
table line                        
Place 33788             0.165576  
      40860             0.021800  
      40340             0.028403  
      40990             0.034570  
      40522             0.229457  
      39742             0.023491  
      28900             0.029785  
      34984             0.133867  
      43928             0.051603  
      25546             0.015422  
      41354             0.088166  
      33268             0.026220  
      29732             0.064942  
      37233             0.018240  
      40899             0.019937  
      39651             0.022285  
      40912             0.018107  
      35400             0.003415  
      41042             0.014166  
      39157             0.043775  
      40756             0.022622  
      37740             0.007309  
      43148             0.041918  
      30915             0.079529  
      36284             0.028373  
      25130             0.063755  
      40457             0.020787  
      43720             0.029581  
      35309             0.009326  
      40795             0.019487

In [11]:
dg = df[['name', 'geosubtype', 'racial_disp_asn_est', 'racial_disp_asn_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_asn_est'])[:30]

name geosubtype  racial_disp_asn_est  \
table line                                                   
Place 42407      Sunnyvale    charter            -0.127184   
      40925    Santa Clara    charter            -0.115677   
      25247         Albany    charter            -0.086147   
      33346         Irvine    charter            -0.081733   
      40873      San Ramon    charter            -0.081285   
      37077  Mountain View    charter            -0.075435   
      43005       Torrance    charter            -0.062877   
      25702        Arcadia    charter            -0.059061   
      29407        Cypress    charter            -0.053302   
      26443       Berkeley    charter            -0.051543   
      38260      Palo Alto    charter            -0.050138   
      31253         Folsom    charter            -0.049376   
      25286       Alhambra    charter            -0.044811   
      28198       Cerritos    charter            -0.034970   
      40665       San Jose    charter            -0.030903   
      40795      San Mateo    charter            -0.027893   
      38442       Pasadena    charter            -0.027117   
      27249        Burbank    charter            -0.026942   
      40704    San Leandro    charter            -0.026163   
      35348   Los Alamitos    charter            -0.024667   
      25221        Alameda    charter            -0.023694   
      35192     Loma Linda    charter            -0.023642   
      40535      San Diego    charter            -0.023313   
      40951     Santa Cruz    charter            -0.022747   
      29342    Culver City    charter            -0.022726   
      42706    Temple City    charter            -0.022329   
      40587  San Francisco    charter            -0.019892   
      39651   Redwood City    charter            -0.018458   
      40158      Roseville    charter            -0.018436   
      27223     Buena Park    charter            -0.018096   

             racial_disp_asn_moe  
table line                        
Place 42407             0.021620  
      40925             0.020381  
      25247             0.039714  
      33346             0.017350  
      40873             0.023824  
      37077             0.022880  
      43005             0.015555  
      25702             0.042091  
      29407             0.027648  
      26443             0.013401  
      38260             0.023035  
      31253             0.014123  
      25286             0.024529  
      28198             0.032812  
      40665             0.005762  
      40795             0.017283  
      38442             0.012835  
      27249             0.014064  
      40704             0.023997  
      35348             0.034013  
      25221             0.021115  
      35192             0.053455  
      40535             0.003863  
      40951             0.010571  
      29342             0.024474  
      42706             0.043402  
      40587             0.003336  
      39651             0.014434  
      40158             0.009149  
      27223             0.033414

In [12]:
dg = df[['name', 'geosubtype', 'racial_disp_wht_est', 'racial_disp_wht_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_wht_est'], ascending=False)[:30]

name geosubtype  racial_disp_wht_est  \
table line                                                        
Place 33788           King City    charter             0.153601   
      40860          San Rafael    charter             0.131725   
      42407           Sunnyvale    charter             0.116974   
      40990         Santa Maria    charter             0.110709   
      37077       Mountain View    charter             0.110376   
      40340             Salinas    charter             0.098253   
      39651        Redwood City    charter             0.097394   
      40925         Santa Clara    charter             0.093330   
      37233                Napa    charter             0.093065   
      40522           Sand City    charter             0.089831   
      41042          Santa Rosa    charter             0.087561   
      43928         Watsonville    charter             0.087034   
      40912       Santa Barbara    charter             0.086687   
      25247              Albany    charter             0.085032   
      34984             Lindsay    charter             0.084473   
      41354             Shafter    charter             0.079566   
      25546             Anaheim    charter             0.077230   
      40795           San Mateo    charter             0.076366   
      39157         Porterville    charter             0.073077   
      40756          San Marcos    charter             0.072621   
      29732  Desert Hot Springs    charter             0.071871   
      40873           San Ramon    charter             0.067986   
      30915              Exeter    charter             0.064927   
      35400         Los Angeles    charter             0.064638   
      43720               Vista    charter             0.063821   
      33346              Irvine    charter             0.062509   
      38624            Petaluma    charter             0.062127   
      43148              Tulare    charter             0.061919   
      31890              Gilroy    charter             0.061505   
      27223          Buena Park    charter             0.054729   

             racial_disp_wht_moe  
table line                        
Place 33788             0.080823  
      40860             0.033753  
      42407             0.022954  
      40990             0.025511  
      37077             0.031314  
      40340             0.017141  
      39651             0.030559  
      40925             0.023151  
      37233             0.020718  
      40522             0.403387  
      41042             0.019641  
      43928             0.033577  
      40912             0.028003  
      25247             0.056007  
      34984             0.062307  
      41354             0.035070  
      25546             0.013719  
      40795             0.019850  
      39157             0.040524  
      40756             0.031051  
      29732             0.060614  
      40873             0.026194  
      30915             0.096960  
      35400             0.002619  
      43720             0.029541  
      33346             0.018802  
      38624             0.030687  
      43148             0.033968  
      31890             0.039225  
      27223             0.025652

In [13]:
dg = df[df['name'] == 'King City']
dg[['prop_adu_his_est', 'prop_adu_his_moe', 'prop_dvap_his_est', 'prop_dvap_his_moe', 
    'prop_adu_dvap_est', 'prop_adu_dvap_moe', 'prop_his_adu_dvap_est', 'prop_his_adu_dvap_moe', 
    'adu_est', 'cvap_est', 'his_adu_est', 'his_adu_moe', 'his_cvap_est', 'his_cvap_moe']]

,,prop_adu_his_est,prop_adu_his_moe,prop_dvap_his_est,prop_dvap_his_moe,prop_adu_dvap_est,prop_adu_dvap_moe,prop_his_adu_dvap_est,prop_his_adu_dvap_moe,adu_est,cvap_est,his_adu_est,his_adu_moe,his_cvap_est,his_cvap_moe
table,line,,,,,,,,,,,,,,
Place,33788,0.852353,0.065886,0.98338,0.175988,0.585722,0.061065,0.675761,0.061603,9245,3830,7880,469,2555,461
